# Gesture Recognition : CNN & LSTM/GRU Models
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

In [4]:
print(len(train_doc))
print(len(val_doc))

663
100


In [5]:
def standardize(img):
    mean = np.mean(img)
    std = np.std(img)
    img = (img - mean) / std
    return img

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,21,22,23,24,25,26,27,28,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,25,80,80,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(80,80)).astype(np.float32)
                    else:
                        image = imresize(image[:,20:340,:],(80,80)).astype(np.float32)
                        
                    batch_data[folder,idx,:,:,0] = standardize(image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = standardize(image[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = standardize(image[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,25,80,80,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(80,80)).astype(np.float32)
                    else:
                        image = imresize(image[:,20:340,:],(80,80)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = standardize(image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = standardize(image[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = standardize(image[:,:,2]) #normalise and feed in the image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 35 # choose the number of epochs
print ('# epochs =', num_epochs)
batch_size = 40 #experiment with the batch size

# training sequences = 663
# validation sequences = 100
# epochs = 35


## Model 1: CONVO 2D + LSTM

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [176]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from keras.layers.recurrent import LSTM

#write your model here
model = Sequential()

model.add(TimeDistributed(Conv2D(64, (3,3),strides=(1,1),padding='same', activation='relu'), input_shape=(18,80,80,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [178]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_200 (TimeDi (None, 18, 80, 80, 64)    1792      
_________________________________________________________________
time_distributed_201 (TimeDi (None, 18, 80, 80, 64)    256       
_________________________________________________________________
time_distributed_202 (TimeDi (None, 18, 40, 40, 64)    0         
_________________________________________________________________
time_distributed_203 (TimeDi (None, 18, 40, 40, 128)   73856     
_________________________________________________________________
time_distributed_204 (TimeDi (None, 18, 40, 40, 128)   512       
_________________________________________________________________
time_distributed_205 (TimeDi (None, 18, 20, 20, 128)   0         
_________________________________________________________________
time_distributed_206 (TimeDi (None, 18, 20, 20, 256)   295168    
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [179]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [180]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [181]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [182]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train ; batch size = 40
 Project_data/val ; batch size = 40
Epoch 1/35


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/

17/17 [==============================] - 63s 4s/step - loss: 1.7397 - categorical_accuracy: 0.1877 - val_loss: 1.5362 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2020-02-0115_11_02.516800/model-00001-1.73958-0.18703-1.53619-0.29000.h5
Epoch 2/35
17/17 [==============================] - 40s 2s/step - loss: 1.6657 - categorical_accuracy: 0.2216 - val_loss: 1.4423 - val_categorical_accuracy: 0.4200

Epoch 00002: saving model to model_init_2020-02-0115_11_02.516800/model-00002-1.66641-0.22172-1.44234-0.42000.h5
Epoch 3/35
17/17 [==============================] - 48s 3s/step - loss: 1.5669 - categorical_accuracy: 0.2914 - val_loss: 1.3805 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_init_2020-02-0115_11_02.516800/model-00003-1.56680-0.29110-1.38052-0.49000.h5
Epoch 4/35
17/17 [==============================] - 48s 3s/step - loss: 1.5361 - categorical_accuracy: 0.3316 - val_loss: 1.3237 - val_categorical_accuracy: 0.5000

Epoch 00004:

17/17 [==============================] - 48s 3s/step - loss: 0.8889 - categorical_accuracy: 0.7096 - val_loss: 0.9747 - val_categorical_accuracy: 0.6400

Epoch 00030: saving model to model_init_2020-02-0115_11_02.516800/model-00030-0.89314-0.70890-0.97474-0.64000.h5
Epoch 31/35
17/17 [==============================] - 47s 3s/step - loss: 0.8727 - categorical_accuracy: 0.6908 - val_loss: 0.9664 - val_categorical_accuracy: 0.6400

Epoch 00031: saving model to model_init_2020-02-0115_11_02.516800/model-00031-0.87140-0.69382-0.96642-0.64000.h5
Epoch 32/35
17/17 [==============================] - 48s 3s/step - loss: 0.9078 - categorical_accuracy: 0.6853 - val_loss: 0.9634 - val_categorical_accuracy: 0.6200

Epoch 00032: saving model to model_init_2020-02-0115_11_02.516800/model-00032-0.90418-0.68929-0.96341-0.62000.h5
Epoch 33/35
17/17 [==============================] - 47s 3s/step - loss: 0.8547 - categorical_accuracy: 0.7229 - val_loss: 0.9648 - val_categorical_accuracy: 0.6400

Epoch 000

## CNN 2D-LSTM : Model 2

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)
batch_size = 40 #experiment with the batch size

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from keras.layers.recurrent import LSTM

#write your model here
model = Sequential()

model.add(TimeDistributed(Conv2D(64, (3,3),strides=(1,1),padding='same', activation='relu'), input_shape=(18,80,80,3)))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))

In [11]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 18, 80, 80, 64)    1792      
_________________________________________________________________
time_distributed_9 (TimeDist (None, 18, 40, 40, 64)    0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 18, 40, 40, 128)   73856     
_________________________________________________________________
time_distributed_11 (TimeDis (None, 18, 20, 20, 128)   0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 18, 20, 20, 256)   295168    
_________________________________________________________________
time_distributed_13 (TimeDis (None, 18, 10, 10, 256)   0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 18, 25600)         0         
__________

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [13]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/

17/17 [==============================] - 141s 8s/step - loss: 1.6478 - categorical_accuracy: 0.1869 - val_loss: 1.5924 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2020-02-0205_53_25.829002/model-00001-1.64973-0.18401-1.59238-0.29000.h5
Epoch 2/10
17/17 [==============================] - 44s 3s/step - loss: 1.6054 - categorical_accuracy: 0.2355 - val_loss: 1.5665 - val_categorical_accuracy: 0.3600

Epoch 00002: saving model to model_init_2020-02-0205_53_25.829002/model-00002-1.60547-0.23379-1.56652-0.36000.h5
Epoch 3/10
17/17 [==============================] - 48s 3s/step - loss: 1.5767 - categorical_accuracy: 0.2711 - val_loss: 1.5438 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-02-0205_53_25.829002/model-00003-1.57605-0.27451-1.54379-0.40000.h5
Epoch 4/10
17/17 [==============================] - 48s 3s/step - loss: 1.5498 - categorical_accuracy: 0.3072 - val_loss: 1.5071 - val_categorical_accuracy: 0.4300

Epoch 00004

In [16]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)
batch_size = 40 #experiment with the batch size

# training sequences = 663
# validation sequences = 100
# epochs = 30


## CNN 2D LSTM : Model 3

In [17]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from keras.layers.recurrent import LSTM

#write your model here
model = Sequential()

model.add(TimeDistributed(Conv2D(64, (3,3),strides=(1,1),padding='same', activation='relu'), input_shape=(18,80,80,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        

model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))

In [18]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 18, 80, 80, 64)    1792      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 18, 80, 80, 64)    256       
_________________________________________________________________
time_distributed_13 (TimeDis (None, 18, 40, 40, 64)    0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 18, 40, 40, 128)   73856     
_________________________________________________________________
time_distributed_15 (TimeDis (None, 18, 40, 40, 128)   512       
_________________________________________________________________
time_distributed_16 (TimeDis (None, 18, 20, 20, 128)   0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 18, 20, 20, 256)   295168    
__________

In [19]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [20]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/

17/17 [==============================] - 149s 9s/step - loss: 1.6868 - categorical_accuracy: 0.2166 - val_loss: 1.4799 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init_2020-02-0214_45_28.830441/model-00001-1.68662-0.21870-1.47991-0.35000.h5
Epoch 2/30
17/17 [==============================] - 46s 3s/step - loss: 1.5580 - categorical_accuracy: 0.2924 - val_loss: 1.3163 - val_categorical_accuracy: 0.5300

Epoch 00002: saving model to model_init_2020-02-0214_45_28.830441/model-00002-1.55905-0.29110-1.31634-0.53000.h5
Epoch 3/30
17/17 [==============================] - 49s 3s/step - loss: 1.4542 - categorical_accuracy: 0.3736 - val_loss: 1.2739 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_init_2020-02-0214_45_28.830441/model-00003-1.45831-0.37104-1.27390-0.49000.h5
Epoch 4/30
17/17 [==============================] - 49s 3s/step - loss: 1.3895 - categorical_accuracy: 0.4147 - val_loss: 1.1916 - val_categorical_accuracy: 0.6100

Epoch 00004


Epoch 00028: saving model to model_init_2020-02-0214_45_28.830441/model-00028-0.59573-0.79638-1.05077-0.61000.h5
Epoch 29/30
17/17 [==============================] - 50s 3s/step - loss: 0.5734 - categorical_accuracy: 0.8254 - val_loss: 1.0511 - val_categorical_accuracy: 0.6100

Epoch 00029: saving model to model_init_2020-02-0214_45_28.830441/model-00029-0.56945-0.82956-1.05114-0.61000.h5
Epoch 30/30
17/17 [==============================] - 50s 3s/step - loss: 0.5987 - categorical_accuracy: 0.8064 - val_loss: 1.0519 - val_categorical_accuracy: 0.6100

Epoch 00030: saving model to model_init_2020-02-0214_45_28.830441/model-00030-0.60051-0.80694-1.05189-0.61000.h5


## CNN 2D GRU : Model 4

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40 # choose the number of epochs
print ('# epochs =', num_epochs)
batch_size = 32 #experiment with the batch size

# training sequences = 663
# validation sequences = 100
# epochs = 40


In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from keras.layers.recurrent import LSTM

#write your model here
model = Sequential()

model.add(TimeDistributed(Conv2D(64, (7,7),strides=(1,1),padding='same', activation='relu'), input_shape=(25,80,80,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(GRU(256, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))

In [12]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_17 (TimeDis (None, 25, 80, 80, 64)    9472      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 25, 80, 80, 64)    256       
_________________________________________________________________
time_distributed_19 (TimeDis (None, 25, 40, 40, 64)    0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 25, 40, 40, 128)   73856     
_________________________________________________________________
time_distributed_21 (TimeDis (None, 25, 40, 40, 128)   512       
_________________________________________________________________
time_distributed_22 (TimeDis (None, 25, 20, 20, 128)   0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 25, 20, 20, 128)   147584    
__________

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [14]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/21 [=>............................] - ETA: 2:56 - loss: 2.1483 - categorical_accuracy: 0.1562

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 132s 6s/step - loss: 1.9096 - categorical_accuracy: 0.2233 - val_loss: 1.6384 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2020-02-0713_58_04.124502/model-00001-1.91215-0.22172-1.63836-0.30000.h5
Epoch 2/40
21/21 [==============================] - 61s 3s/step - loss: 1.7354 - categorical_accuracy: 0.2704 - val_loss: 1.5451 - val_categorical_accuracy: 0.3300

Epoch 00002: saving model to model_init_2020-02-0713_58_04.124502/model-00002-1.73754-0.26998-1.54510-0.33000.h5
Epoch 3/40
21/21 [==============================] - 66s 3s/step - loss: 1.5708 - categorical_accuracy: 0.3592 - val_loss: 1.4535 - val_categorical_accuracy: 0.3600

Epoch 00003: saving model to model_init_2020-02-0713_58_04.124502/model-00003-1.57193-0.36048-1.45347-0.36000.h5
Epoch 4/40
21/21 [==============================] - 66s 3s/step - loss: 1.4853 - categorical_accuracy: 0.3761 - val_loss: 1.2544 - val_categorical_accuracy: 0.4600

Epoch 00004


Epoch 00028: saving model to model_init_2020-02-0713_58_04.124502/model-00028-0.51675-0.85219-1.06029-0.60000.h5
Epoch 29/40
21/21 [==============================] - 64s 3s/step - loss: 0.4903 - categorical_accuracy: 0.8493 - val_loss: 1.0616 - val_categorical_accuracy: 0.6000

Epoch 00029: saving model to model_init_2020-02-0713_58_04.124502/model-00029-0.48754-0.85068-1.06162-0.60000.h5
Epoch 30/40
21/21 [==============================] - 65s 3s/step - loss: 0.5129 - categorical_accuracy: 0.8375 - val_loss: 1.0618 - val_categorical_accuracy: 0.6000

Epoch 00030: saving model to model_init_2020-02-0713_58_04.124502/model-00030-0.51341-0.83710-1.06176-0.60000.h5
Epoch 31/40
21/21 [==============================] - 64s 3s/step - loss: 0.5123 - categorical_accuracy: 0.8202 - val_loss: 1.0626 - val_categorical_accuracy: 0.6000

Epoch 00031: saving model to model_init_2020-02-0713_58_04.124502/model-00031-0.51318-0.81900-1.06259-0.60000.h5
Epoch 32/40
21/21 [==============================

## CNN 2D GRU model 5

In [20]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from keras.layers.recurrent import LSTM

#write your model here
model = Sequential()

model.add(TimeDistributed(Conv2D(64, (7,7),strides=(1,1),padding='same', activation='relu'), input_shape=(25,80,80,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(Dropout(0.25))       
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(GRU(256, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))

In [21]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_55 (TimeDis (None, 25, 80, 80, 64)    9472      
_________________________________________________________________
time_distributed_56 (TimeDis (None, 25, 80, 80, 64)    256       
_________________________________________________________________
time_distributed_57 (TimeDis (None, 25, 40, 40, 64)    0         
_________________________________________________________________
time_distributed_58 (TimeDis (None, 25, 40, 40, 128)   73856     
_________________________________________________________________
time_distributed_59 (TimeDis (None, 25, 40, 40, 128)   512       
_________________________________________________________________
time_distributed_60 (TimeDis (None, 25, 20, 20, 128)   0         
_________________________________________________________________
time_distributed_61 (TimeDis (None, 25, 20, 20, 128)   147584    
__________

In [22]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/21 [===>..........................] - ETA: 1:28 - loss: 2.0577 - categorical_accuracy: 0.1875

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 76s 4s/step - loss: 1.8877 - categorical_accuracy: 0.2042 - val_loss: 1.6784 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-02-0713_58_04.124502/model-00001-1.88810-0.20513-1.67841-0.25000.h5
Epoch 2/40
21/21 [==============================] - 65s 3s/step - loss: 1.7607 - categorical_accuracy: 0.2626 - val_loss: 1.7184 - val_categorical_accuracy: 0.2700

Epoch 00002: saving model to model_init_2020-02-0713_58_04.124502/model-00002-1.76352-0.26094-1.71838-0.27000.h5
Epoch 3/40
21/21 [==============================] - 66s 3s/step - loss: 1.7030 - categorical_accuracy: 0.2788 - val_loss: 1.5229 - val_categorical_accuracy: 0.3900

Epoch 00003: saving model to model_init_2020-02-0713_58_04.124502/model-00003-1.70103-0.27903-1.52287-0.39000.h5
Epoch 4/40
21/21 [==============================] - 66s 3s/step - loss: 1.5555 - categorical_accuracy: 0.3745 - val_loss: 1.3734 - val_categorical_accuracy: 0.4700

Epoch 00004:


Epoch 00028: saving model to model_init_2020-02-0713_58_04.124502/model-00028-1.01329-0.61086-1.24751-0.52000.h5
Epoch 29/40
21/21 [==============================] - 65s 3s/step - loss: 1.0178 - categorical_accuracy: 0.6187 - val_loss: 1.2480 - val_categorical_accuracy: 0.5100

Epoch 00029: saving model to model_init_2020-02-0713_58_04.124502/model-00029-1.01803-0.61991-1.24802-0.51000.h5
Epoch 30/40
21/21 [==============================] - 64s 3s/step - loss: 1.0417 - categorical_accuracy: 0.5695 - val_loss: 1.2482 - val_categorical_accuracy: 0.5200

Epoch 00030: saving model to model_init_2020-02-0713_58_04.124502/model-00030-1.03822-0.57014-1.24818-0.52000.h5
Epoch 31/40
21/21 [==============================] - 65s 3s/step - loss: 1.0453 - categorical_accuracy: 0.5759 - val_loss: 1.2472 - val_categorical_accuracy: 0.5200

Epoch 00031: saving model to model_init_2020-02-0713_58_04.124502/model-00031-1.04210-0.57768-1.24717-0.52000.h5
Epoch 32/40
21/21 [==============================